# Conversation Memory for LLMs
Q: Do the GTP models from OpenAI retain memory of previous prompts and incorporate them into subsequent responses?

A: The GPT models are stateless, meaning the gpt model doesn't remember past prompts. Each time we interact with it, we have to provide all the necessary information again. Currently, all large language models work this way—they need the full set of instructions every time.

In [24]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# Load the .env
load_dotenv(find_dotenv())
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

## Without conversation history/memory: 
With our current helper function. The model will not remember previous prompts.

In [25]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [26]:
print("Response: ", get_completion("My name is Aaron"))

Response:  Nice to meet you, Aaron! How can I assist you today?


In [27]:
print("Response: " , get_completion("What's my name?"))

Response:  I'm sorry, I do not have access to personal information such as your name.


## With conversation history/memory: 
Modify the helper function to help the model remember previous prompts and responses.

In [28]:
import json # This will be used to format the response into JSON format for better readability.

def get_completion(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    # Append AI response to the messages list
    messages.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })
    return response.choices[0].message.content

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    }
]

When OpenAI replies, we add that to messages array.

In [29]:
# Example usage
prompt = "My name is Aaron"
messages.append({"role": "user", "content": prompt})

print("Prompt:", prompt)
print("Response:", get_completion(messages))


Prompt: My name is Aaron
Response: Nice to meet you, Aaron! How can I assist you today?


In [30]:
# See the response in better format.
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "My name is Aaron"
    },
    {
        "role": "assistant",
        "content": "Nice to meet you, Aaron! How can I assist you today?"
    }
]


When making another request, we send the whole array back

In [31]:
prompt = "What is my name?"
messages.append({"role": "user", "content": prompt})

print("Prompt:", prompt)
print("Response:", get_completion(messages))

Prompt: What is my name?
Response: Your name is Aaron.


In [32]:
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "My name is Aaron"
    },
    {
        "role": "assistant",
        "content": "Nice to meet you, Aaron! How can I assist you today?"
    },
    {
        "role": "user",
        "content": "What is my name?"
    },
    {
        "role": "assistant",
        "content": "Your name is Aaron."
    }
]
